In [1]:
import pandas as pd
from IPython.display import display
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

dir_suffix = "data/"
dir_suffix_entpackt = dir_suffix + "Daten_ausgepackt/"

In [2]:
print("Tip25W Testdaten Template:")
testdata = pd.read_csv(dir_suffix+"tip25W_testdaten_template.csv")

display(testdata.head(12))

print("Tip25W Trainingsdaten:")
trainingdata = pd.read_csv(dir_suffix+"tip25W_trainingsdaten.csv")
# ist das gleiche wie orders gefiltert nach nur prior und das gleiche wie der prior Datensatz
#trainingdata = trainingdata[:300000]
display(trainingdata.head(11))

print("Prior:")
df_orders_prior = pd.read_csv(dir_suffix_entpackt+ "order_products__prior.csv")
display(df_orders_prior.head(10))

print("train:")
df_orders_train = pd.read_csv(dir_suffix_entpackt + "order_products__train.csv")
display(df_orders_train)

print("orders")
df_orders = pd.read_csv(dir_suffix_entpackt + "orders.csv")
display(df_orders.head(10))

print("products:")
df_products = pd.read_csv(dir_suffix_entpackt + "products.csv")
display(df_products.head(10))



Tip25W Testdaten Template:


,Unnamed: 0,order_id,tip,tip_pc
0,10,1187899,NaN,NaN
1,25,1492625,NaN,NaN
2,49,2196797,NaN,NaN
3,74,525192,NaN,NaN
4,78,880375,NaN,NaN
5,82,1094988,NaN,NaN
6,88,1822501,NaN,NaN
7,115,1827621,NaN,NaN
8,129,2316178,NaN,NaN
9,200,2180313,NaN,NaN


Tip25W Trainingsdaten:


,Unnamed: 0,order_id,tip,tip_pc
0,0,2539329,True,0.092177
1,1,2398795,False,0.000000
2,2,473747,False,0.000000
3,3,2254736,True,0.102231
4,4,431534,True,0.092990
5,5,3367565,False,0.000000
6,6,550135,False,0.000000
7,7,3108588,False,0.000000
8,8,2295261,False,0.000000
9,9,2550362,False,0.000000


Prior:


...

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


orders


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


products:


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
5,6,Dry Nose Oil,11,11
6,7,Pure Coconut Water With Orange,98,7
7,8,Cut Russet Potatoes Steam N' Mash,116,1
8,9,Light Strawberry Blueberry Yogurt,120,16
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7


In [3]:

merged = trainingdata.merge(df_orders_prior, on="order_id", how="left")
merged = merged.merge(df_orders, on="order_id", how="left")

merged = merged.fillna(0)

#order_merged = order_merged.merge(df_products, on="product_id", how="left")
display(merged.head(5))
print(merged["eval_set"].unique())

,Unnamed: 0,order_id,tip,tip_pc,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,0,2539329,True,0.092177,196,1,0,1,prior,1,2,8,0.0
1,0,2539329,True,0.092177,14084,2,0,1,prior,1,2,8,0.0
2,0,2539329,True,0.092177,12427,3,0,1,prior,1,2,8,0.0
3,0,2539329,True,0.092177,26088,4,0,1,prior,1,2,8,0.0
4,0,2539329,True,0.092177,26405,5,0,1,prior,1,2,8,0.0


['prior']


In [ ]:

order_features = (
    df_orders_prior
    .groupby("order_id")
    .agg(
        num_products=("product_id", "count"),
        num_reordered=("reordered", "sum"),
        avg_add_to_cart=("add_to_cart_order", "mean"),
        max_add_to_cart=("add_to_cart_order", "max"),
    )
    .reset_index()
)

prior_with_dept = df_orders_prior.merge(
    df_products[["product_id", "department_id"]],
    on="product_id",
    how="left"
)

dept_counts = (
    prior_with_dept
    .groupby(["order_id", "department_id"])
    .size()
    .unstack(fill_value=0)
)

dept_counts.columns = [f"department_id_{int(c)}" for c in dept_counts.columns]
dept_counts = dept_counts.reset_index()

order_features = order_features.merge(
    dept_counts,
    on="order_id",
    how="left"
)

order_features = order_features.merge(
    df_orders,
    on="order_id",
    how="left"
)

final_df = order_features.merge(
    trainingdata[["order_id", "tip", "tip_pc"]],
    on="order_id",
    how="left"
)

df_day_encoded = pd.get_dummies(final_df, columns=["order_dow"], prefix="dow",drop_first=True)
df_day_encoded

df_encoded_final = pd.get_dummies(df_day_encoded, columns=["order_hour_of_day"], prefix="hour",drop_first=True)
df_encoded_final
df_encoded_final.head(10)


,order_id,num_products,num_reordered,avg_add_to_cart,max_add_to_cart,department_id_1,department_id_2,department_id_3,department_id_4,department_id_5,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,2,9,6,5.0,9,0,0,0,3,0,...,False,False,False,False,False,False,False,False,False,False
1,3,8,8,4.5,8,0,0,1,3,0,...,False,False,False,True,False,False,False,False,False,False
2,4,13,12,7.0,13,0,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
3,5,26,21,13.5,26,0,0,0,7,0,...,False,False,True,False,False,False,False,False,False,False
4,6,3,0,2.0,3,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
5,7,2,0,1.5,2,1,0,0,0,0,...,True,False,False,False,False,False,False,False,False,False
6,8,1,1,1.0,1,0,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
7,9,15,10,8.0,15,0,0,2,4,0,...,False,False,False,False,False,True,False,False,False,False
8,10,15,8,8.0,15,0,0,0,10,0,...,False,False,False,False,False,False,False,False,False,False
9,11,5,5,3.0,5,1,0,0,0,0,...,False,False,False,False,False,True,False,False,False,False


In [5]:
df = df_encoded_final.copy()
df = df.sort_values(["user_id", "order_number"]).reset_index(drop=True)

tip = df["tip"].astype(int)
df["tips_cum"] = tip.groupby(df["user_id"]).cumsum()
df["tips_past"] = df["tips_cum"] - tip
df["orders_past"] = df["order_number"] - 1
df["tip_rate_past"] = (df["tips_past"] / df["orders_past"]).where(df["orders_past"] > 0, 0.0)
df.head(20)

,order_id,num_products,num_reordered,avg_add_to_cart,max_add_to_cart,department_id_1,department_id_2,department_id_3,department_id_4,department_id_5,...,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,tips_cum,tips_past,orders_past,tip_rate_past
0,2539329,5,0,3.0,5,0,0,0,0,0,...,False,False,False,False,False,False,1,0,0,0.000000
1,2398795,6,3,3.5,6,0,0,0,1,0,...,False,False,False,False,False,False,1,1,1,1.000000
2,473747,5,3,3.0,5,0,0,0,0,0,...,False,False,False,False,False,False,1,1,2,0.500000
3,2254736,5,5,3.0,5,0,0,0,0,0,...,False,False,False,False,False,False,2,1,3,0.333333
4,431534,8,5,4.5,8,0,0,0,4,0,...,False,False,False,False,False,False,3,2,4,0.500000
5,3367565,4,4,2.5,4,0,0,0,0,0,...,False,False,False,False,False,False,3,3,5,0.600000
6,550135,5,5,3.0,5,0,0,0,0,0,...,False,False,False,False,False,False,3,3,6,0.500000
7,3108588,6,4,3.5,6,0,0,0,0,0,...,False,False,False,False,False,False,3,3,7,0.428571
8,2295261,6,6,3.5,6,0,0,0,0,0,...,False,False,False,False,False,False,3,3,8,0.375000
9,2550362,9,6,5.0,9,0,0,0,0,0,...,False,False,False,False,False,False,3,3,9,0.333333


In [6]:
df = df.drop(["order_id", "avg_add_to_cart","max_add_to_cart", "tips_cum", "tips_past","orders_past", "user_id","eval_set"],axis=1)
display(df)

,num_products,num_reordered,department_id_1,department_id_2,department_id_3,department_id_4,department_id_5,department_id_6,department_id_7,department_id_8,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,tip_rate_past
0,5,0,0,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,False,0.000000
1,6,3,0,0,0,1,0,0,1,0,...,False,False,False,False,False,False,False,False,False,1.000000
2,5,3,0,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,False,0.500000
3,5,5,0,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,False,0.333333
4,8,5,0,0,0,4,0,0,1,0,...,True,False,False,False,False,False,False,False,False,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3214869,3,3,0,0,0,0,0,0,1,0,...,True,False,False,False,False,False,False,False,False,0.250000
3214870,9,3,1,0,1,1,0,0,1,0,...,False,False,False,True,False,False,False,False,False,0.333333
3214871,8,6,0,0,1,1,0,0,1,0,...,False,False,False,False,False,False,False,False,False,0.300000
3214872,20,14,0,0,2,5,0,0,1,0,...,False,False,False,False,False,False,False,False,False,0.272727


In [7]:
dept_cols = [c for c in df.columns if c.startswith("department_id_")]

display(dept_cols)

dep_tip_count = (
    df[dept_cols].gt(0)
    .astype(int)
    .sum()
)

avg_tip_per_dept = (
    df[dept_cols].gt(0)
    .astype(int)
    .multiply(df["tip_pc"], axis=0)
    .sum()
    /dep_tip_count
)

['department_id_1',
 'department_id_2',
 'department_id_3',
 'department_id_4',
 'department_id_5',
 'department_id_6',
 'department_id_7',
 'department_id_8',
 'department_id_9',
 'department_id_10',
 'department_id_11',
 'department_id_12',
 'department_id_13',
 'department_id_14',
 'department_id_15',
 'department_id_16',
 'department_id_17',
 'department_id_18',
 'department_id_19',
 'department_id_20',
 'department_id_21']

In [8]:
display(merged.head(1))

order_df = df

,Unnamed: 0,order_id,tip,tip_pc,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,0,2539329,True,0.092177,196,1,0,1,prior,1,2,8,0.0


In [9]:



#X["is_first_order"] = (X["order_number"] == 1).astype(int)
#X["days_since_prior_order"] = X["days_since_prior_order"].fillna(0)
#X.head(10)

order_df.head(3)


single = order_df.tail(1) 
order_df =  order_df.iloc[:-1].copy() 

order_df = order_df.fillna(0)



In [10]:
X = order_df.drop(columns="tip_pc")




y = order_df["tip_pc"]


X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,      # 20 % Testdaten
    random_state=42     # reproduzierbar
)



In [11]:
X_train.head(3)
X_test

,num_products,num_reordered,department_id_1,department_id_2,department_id_3,department_id_4,department_id_5,department_id_6,department_id_7,department_id_8,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,tip_rate_past
737360,3,3,0,0,1,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0.456790
1215397,5,4,0,0,3,0,0,0,0,0,...,True,False,False,False,False,False,False,False,False,0.769231
2209486,10,9,0,0,0,6,1,0,0,0,...,False,False,False,False,False,False,False,False,False,1.000000
778849,19,16,2,0,1,11,0,0,1,0,...,True,False,False,False,False,False,False,False,False,1.000000
621365,14,14,0,0,0,3,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0.735294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1789837,9,6,0,0,1,4,0,0,1,0,...,False,False,False,False,False,False,False,False,False,0.464286
1293193,39,36,0,0,1,1,0,0,8,0,...,False,False,False,False,False,False,False,False,False,0.724138
2885702,24,15,2,0,0,7,0,1,4,0,...,False,False,False,True,False,False,False,False,False,0.000000
1728032,29,29,1,0,2,7,0,0,4,0,...,False,False,False,False,False,False,False,False,False,0.846154


In [12]:
numeric_features = [
    "tip",
    "product_id",
    "add_to_cart_order",
    "reordered",
    "user_id",
    "order_number",
    "order_dow",
    "order_hour_of_day"
    
]
categorical_features = [
    "product_name"
]


""" preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features)
    ]
) """


' preprocessor = ColumnTransformer(\n    transformers=[\n        ("num", StandardScaler(), numeric_features)\n    ]\n) '

In [13]:

order_df.head(5)
X_test

,num_products,num_reordered,department_id_1,department_id_2,department_id_3,department_id_4,department_id_5,department_id_6,department_id_7,department_id_8,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,tip_rate_past
737360,3,3,0,0,1,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0.456790
1215397,5,4,0,0,3,0,0,0,0,0,...,True,False,False,False,False,False,False,False,False,0.769231
2209486,10,9,0,0,0,6,1,0,0,0,...,False,False,False,False,False,False,False,False,False,1.000000
778849,19,16,2,0,1,11,0,0,1,0,...,True,False,False,False,False,False,False,False,False,1.000000
621365,14,14,0,0,0,3,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0.735294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1789837,9,6,0,0,1,4,0,0,1,0,...,False,False,False,False,False,False,False,False,False,0.464286
1293193,39,36,0,0,1,1,0,0,8,0,...,False,False,False,False,False,False,False,False,False,0.724138
2885702,24,15,2,0,0,7,0,1,4,0,...,False,False,False,True,False,False,False,False,False,0.000000
1728032,29,29,1,0,2,7,0,0,4,0,...,False,False,False,False,False,False,False,False,False,0.846154


In [14]:
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor


model = LinearRegression()
""" model = HistGradientBoostingRegressor(
    learning_rate=0.05,
    max_depth=6,
    max_iter=300,         # entspricht grob n_estimators
    early_stopping=True,  # stoppt wenn’s nicht mehr besser wird
    random_state=42
) """
pipeline = Pipeline(
    steps=[
        ("preprocessor", StandardScaler()),
        ("model", model)
    ]
)


pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)


display(single)

single_pred = single.drop(["tip_pc"], axis=1)

print(pipeline.predict(single_pred))

print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R²:", r2_score(y_test, y_pred))

,num_products,num_reordered,department_id_1,department_id_2,department_id_3,department_id_4,department_id_5,department_id_6,department_id_7,department_id_8,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,tip_rate_past
3214873,9,4,0,0,0,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0.25


[0.11943891]
MAE: 0.014371169938849713
RMSE: 0.019532369055072746
R²: 0.9117995562641777


In [17]:
X.isna().sum().sort_values(ascending=False)


num_products              0
num_reordered             0
department_id_1           0
department_id_2           0
department_id_3           0
department_id_4           0
department_id_5           0
department_id_6           0
department_id_7           0
department_id_8           0
department_id_9           0
department_id_10          0
department_id_11          0
department_id_12          0
department_id_13          0
department_id_14          0
department_id_15          0
department_id_16          0
department_id_17          0
department_id_18          0
department_id_19          0
department_id_20          0
department_id_21          0
order_number              0
days_since_prior_order    0
tip                       0
dow_0                     0
dow_1                     0
dow_2                     0
dow_3                     0
dow_4                     0
dow_5                     0
dow_6                     0
hour_0                    0
hour_1                    0
hour_2              